<a href="https://colab.research.google.com/github/cicattzo/nlp_project/blob/main/NLP_Pretrained_Summarization_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
%%bash
pip -q install torch
pip -q install transformers
# pip -q install datasets
pip -q install tqdm
pip -q install rouge_score
pip -q install sacrebleu
pip install datasets==1.0.2
pip install sumy
pip install SentencePiece
# pip install transformers==4.0.1

  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=0608f959d4f0567679fb8d2e94425c338ab96a6047d80e81f6793cf285bc6a0d
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21680 sha256=4febb66a2992d0277d589b3c1751e799795d3e5d26a1ffed4f75e6f63fbf8b07
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built pycountry breadability


In [ ]:
%%bash
mkdir "/content/gdrive/MyDrive/6864_project/"
cd "/content/gdrive/MyDrive/6864_project/"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/6864_project/’: No such file or directory
bash: line 2: cd: /content/gdrive/MyDrive/6864_project/: No such file or directory


In [ ]:
MODEL_FOLDER = "/content/gdrive/MyDrive/6864_project/"
import pandas as pd
import torch
import numpy as np
import datasets
import transformers
import random
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from IPython.display import display
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from transformers import EncoderDecoderModel

In [ ]:
test_data_pd = pd.read_csv("bc3_test.csv")
test_data = test_data_pd.to_dict('records')
test_data = [x for x in test_data]

In [ ]:
#intializing the tokenizer and choosing the pretrained model
device = 'cpu'
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

## Evaluating our model using ROUGE and BLEU

In [ ]:
%%bash
pip install sacrebleu
pip install rouge_score

In [ ]:
PAD_INDEX = 0
UNK_INDEX = 1
SOS_INDEX = 2
EOS_INDEX = 3
def greedy_decode(batch_logits):
  '''
  decodes the logits in a greedy way, picks the most probable word till EOS token is found
  logits: tensor, (batch_size x seq_len x vocab_size)
  '''
  batch_out_ids = torch.argmax(batch_logits, dim=2)
  batch_predicted = []
  for i in range(batch_out_ids.shape[0]):
    out_ids = batch_out_ids[i] 
    out_ids_trunc = []
    for id in out_ids:
      if id == EOS_INDEX:
        break
      out_ids_trunc.append(id)
    out_str = ' '.join(tokenizer.batch_decode(torch.stack(out_ids_trunc)))
    batch_predicted.append(out_str)
  return batch_predicted

In [ ]:
import sacrebleu
from rouge_score import rouge_scorer

rscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

In [ ]:
#redefining the keys to the bc3 dataset in case that wasnt trained on
full_text_key = 'body'
label_key = 'summary'
num_test_cases = len(test_data)
batch_size = 1

rouge_scores_list = []
bleu_score_list = []
all_comparisons = []
prev_inputs = None

for i in tqdm(range(num_test_cases), position=0, leave=True):
  # if i > 4:
  #   break

  input_batch  = test_data[i]['body']
  label_batch  = test_data[i]['summary']

  # if prev_inputs == input_batch:
  #   print("skipped")
  #   continue

  if len(input_batch) > 512:
    input_batch = input_batch[:512]
  
  input_ids = tokenizer(input_batch, padding=True, return_tensors="pt").input_ids.to(device)
  label_ids = tokenizer(label_batch, padding=True, return_tensors="pt").input_ids.to(device)

  output = model.generate(
    input_ids, 
    max_length=32, 
    num_beams=5, 
    early_stopping=True
  )


  prev_inputs = input_batch

  actual_output = tokenizer.decode(output[0], skip_special_tokens=True)
  rouge_scores_list.append(rscorer.score(label_batch, actual_output))

  all_comparisons.append([input_batch, actual_output, label_batch])
  # bleu_score_list.extend(sacrebleu.raw_corpus_bleu(label_ids, output[0], .01))

100%|██████████| 201/201 [19:32<00:00,  5.83s/it]


In [ ]:
print('Rouge1', np.mean([sc['rouge1'].fmeasure for sc in rouge_scores_list]))
print('Rouge2', np.mean([sc['rouge2'].fmeasure for sc in rouge_scores_list]))
# print('BLEU', np.mean(bleu_score_list))

Rouge1 0.17430950962403424
Rouge2 0.04525819727015502


In [ ]:
for i in range(len(all_comparisons)):
  print("Inputted email:\n", all_comparisons[i][0])
  print("Predicted Summary:\n", all_comparisons[i][1])
  print("Labelled Summary:\n", all_comparisons[i][2])


Inputted email:
 > In fact, I have it on the shelf behind me at work as I type this, beside my XHTML, XSLT, CSS, usability, and disability references. 
Is this bookshelf documented on Kynn.COM, or would you consider sharing with us a short list of the references you find the most helpful? 
Al
Predicted Summary:
 Is this bookshelf documented on Kynn.COM?
Labelled Summary:
 Al asks if Kynn's book collection is documented on Kynn's site.
Inputted email:
 > Is this bookshelf documented on Kynn.COM, or would you consider sharing with us a short list of the references you find the most helpful? 
Hey, that's not a bad idea. In fact, I encourage everyone else to share their favorite bookshelf collections -- if relevant to WAI activities. I'll post mine on kynn.com tomorrow when I'm back in the office.
Predicted Summary:
 Is this bookshelf documented on Kynn.COM?
Labelled Summary:
 Kynn thinks that Al has a good idea and encourages everyone to share their bookshelf collections if relevant to WA

In [ ]:
actual_output

"Is it acceptable to provide a warning message, something like 'Submitting this form may result in a pop-up dialog?"

In [ ]:
input_batch

"&gt; I have built a form which has a submit button on it. &gt; If the user activates the button and hasn't filled out the form correctly, a javascript 'alert' pop-up is presented to the user, giving more information. &gt; This appears to be in breach of checkpoint 10.1 &gt; http://www.w3.org/TR/WCAG10/wai-pageauth.html#tech-avoid-pop-ups &gt; Would it be acceptable to provide a warning message, something like 'Submitting this form may result in a pop-up dialog being displayed' before the submit button, in t"